In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import re
import pymongo

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.curriculumDB
db.curriculum.delete_many({})
curriculum_collection = db.curriculumDB


In [3]:
#First is HBS

source='https://www.hbs.edu/coursecatalog/indexcourse.html'

html = urlopen(str(source))
soup = BeautifulSoup(html, 'html.parser')

tdlist = soup.find_all('td')

reflist = []
for i in range(len(tdlist)):
    if str(tdlist[i])[:27] == '<td><a href="/coursecatalog':
        reflist.append(str(tdlist[i]))
    
#This code is necessary because HBS has course descriptions on different pages, rather than collected in one URL.

linklist = []
for i in range(len(reflist)):
    linklist.append(reflist[i][14:37])
    
#Creating a list of link extensions    
    
descraw = []

for i in range(len(linklist)):
    link = urlopen('https://www.hbs.edu/'+ linklist[i])
    descsoup = BeautifulSoup(link, 'html.parser')
    descraw.append(descsoup.findAll('p'))
    
#navigating to link extensions and retrieving course descriptions    


titlelist = []
for i in range(len(reflist)):
    titlelist.append(re.sub('<[^<]+?>', '', str(reflist[i]))) 
#titlelist


hbsdf = pd.DataFrame(titlelist, columns=["Course"])
hbsdf['Description'] = descraw

hbsdf['Source'] = source


#This writes everything to a dataframe. Easy to export to excel. The original data is stored in a list, so it could
#easily be written to a different kind of object, but I have used dataframes here for convenience.
    

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [4]:
hbsdf['School'] = 'Harvard Business School'

In [5]:
hbsdf

for i in range(len(hbsdf['Description'])):
    hbsdf['Description'][i]= re.sub('<[^<]+?>', '', str(hbsdf['Description'][i]))
    
    hbsdf['Source'][i] = 'https://www.hbs.edu/'+ linklist[i]

In [6]:
#reflist

In [7]:
hbsdf

,Course,Description,Source,School
0,The Arts of Communication,"[\nCourse Number 7315 (Q1), Course Number 7515...",https://www.hbs.edu/coursecatalog/7315.html,Harvard Business School
1,Authentic Leader Development,"[\nCourse Number 2090, The purpose of ALD is t...",https://www.hbs.edu/coursecatalog/2090.html,Harvard Business School
2,Becoming a General Manager,"[\nCourse Number 1556\n, \nProfessor Amy C. Ed...",https://www.hbs.edu/coursecatalog/1556.html,Harvard Business School
3,Building and Sustaining a Successful Enterprise,"[\nCourse Number 1504\n, \nSenior Lecturer Che...",https://www.hbs.edu/coursecatalog/1504.html,Harvard Business School
4,Business Analysis and Valuation Using Financia...,"[\nCourse Number 1306\n, \nAssociate Professor...",https://www.hbs.edu/coursecatalog/1306.html,Harvard Business School
5,Business at the Base of the Pyramid,"[\nCourse Number 1908\n, \nSenior Lecturer Mic...",https://www.hbs.edu/coursecatalog/1908.html,Harvard Business School
6,"The Business of Entertainment, Media, and Sports","[\nCourse Number 1914\n, \nProfessor Anita Elb...",https://www.hbs.edu/coursecatalog/1914.html,Harvard Business School
7,The Coming of Managerial Capitalism: The Unite...,"[\nCourse Number 1122\n, \nProfessor Tom Nicho...",https://www.hbs.edu/coursecatalog/1122.html,Harvard Business School
8,Contemporary Developing Countries: Entrepreneu...,"[\nCourse Number 1266\n, \nProfessor Tarun Kha...",https://www.hbs.edu/coursecatalog/1266.html,Harvard Business School
9,Corporate Financial Operations,"[\nCourse Number 1416\n, \nProfessor C. Fritz ...",https://www.hbs.edu/coursecatalog/1416.html,Harvard Business School


In [8]:
hbsdict = hbsdf.to_dict(orient='list')



In [9]:
#hbsdict['Description']

In [10]:
re.sub('<[^<]+?>', '', str(hbsdict['Description'][0]))

"[\nCourse Number 7315 (Q1), Course Number 7515 (Q2) and Course Number 7415 (Q3)\n, \nInstructor  Timothy McCarthy (HKS Adjunct Lecturer on Public Policy)\nFall; Q1; 1.5 credits and Spring Q3; 1.5 credits\n  14 Sessions\n  \n  Enrollment limited to 40 students in Q1 and 40 students in Q3\n  Paper, \nInstructor  Larry Quartana\nFall; Q2; 1.5 credits\n  14 Sessions\n  \n  Enrollment limited to 40 students in Q2\n  Paper, Today's leaders must have the ability not only to analyze thoughtfully but also to communicate clearly and persuasively. This course will seek to strengthen the capacity of each student to speak well in public settings while navigating a range of leadership scenarios. Approximately one-third of the course will be devoted to classes that introduce students to strategies of communication and to models of public presentation. The rest of the course will consist of workshops in which students will hone their skills in public speaking. The course is designed for future leader

In [11]:
#db.curriculum.insert_one(hbsdict)

In [12]:
#Next is Wharton. This was the first code that I wrote, so it is a bit clunkier. Also, this includes prerequisites. (Note: this is removed in current mongoDB version)
#I have decided to remove this from the final version, because it is outside the scope of the original project.

source='https://mgmt.wharton.upenn.edu/programs/mba/course-descriptions/'

html = urlopen(str(source))
soup = BeautifulSoup(html, 'html.parser')

full_text = soup.get_text()

title = soup.select('h3')
para = soup.select('p')

for i in range(len(title)):
    title[i] = re.sub('<[^<]+?>', '', str(title[i]))

#you can see here I am scrubbing everything unnecessarily early in the process. 

title1df = pd.DataFrame(title, columns=['Course'])

title1df['Source'] = source

#The above two sections are taking the raw course catalog and putting it into a dataframe. The below sections are removing the
#'prerequisites' columns. Because this comes from the original code, where I created an excel file with only course names, 
#there is some redundancy here. Keeping it in case the code comes in handy later. 

for i in range(len(title1df)):
    if i >0:
        title1df['Source'][i]= ''
    else:
        continue
        
prereq = []

for strong_tag in soup.find_all('strong'):
    prereq.append((strong_tag.text, strong_tag.next_sibling))
    
poplist = []
prereqlist = []

for i in range(len(para)):
    if str(para[i])[:11] == '<p><strong>':
        poplist.append(i)
    else:
        continue

newpara = []

for i in range(len(para)):
    if i in poplist:
        prereqlist.append(para[i])
    else:
        newpara.append(para[i])

        
#All of this is to fix the problem of classes without prerequisites. This is redundant, but the code is included here in 
#case it is useful for another school's formatting (all these websites have different formatting)

fulldf = pd.DataFrame(title[:-3], columns= ['Course'])

fulldf['Source'] = source
fulldf['School'] = 'Wharton'

fulldf['Description'] = ''

for i in range(len(newpara)):
    fulldf['Description'][i]= re.sub('<[^<]+?>', '', str(newpara[i]))

newpara
fulldf['Prerequisites'] = ""       

for i in range(len(fulldf)):
    for j in range(len(poplist)):
        if i == poplist[j]:
            fulldf['Prerequisites'][i]= re.sub('<[^<]+?>', '', str(prereqlist[j]))
        else: continue   
fullcols = fulldf.columns.tolist()

newcols = ['Course','Description',  'Source', 'School']

whartondf = fulldf[newcols]


In [13]:
#whartondict = whartondf.to_dict(orient='list')

#db.curriculum.insert_many([whartondict, hbsdict])

In [14]:
#Stanford had problably the simplest, best-formatted website. Bless the people at Stanford who are running this website. 

source = 'https://exploredegrees.stanford.edu/graduateschoolofbusiness/#courseinventory'

html = urlopen(str(source))
soup = BeautifulSoup(html, 'html.parser')

#soup.find_all()
title = soup.select('strong')

for i in range(len(title)):
    title[i] = re.sub('<[^<]+?>', '', str(title[i]))
    
titlelist = []
desclist = []

for i in soup.findAll('p'):
    if str(i)[:28] == '<p class="courseblocktitle">':
        titlelist.append(i)
    
    elif str(i)[:27] == '<p class="courseblockdesc">':
        desclist.append(i)
        
        
#Below is simply to remove html tags from the text.
        
for i in range(len(title)):
    title[i] = re.sub('<[^<]+?>', '', str(title[i]))     
    
    

stanforddf = pd.DataFrame(titlelist, columns=["Course"])

stanforddf['Description'] = desclist

for i in range(len(stanforddf)):
    stanforddf['Course'][i] = re.sub('<[^<]+?>', '', str( stanforddf['Course'][i]))
    stanforddf['Description'][i] = re.sub('<[^<]+?>', '', str( stanforddf['Description'][i]))
    


stanforddf['Source'] = source
stanforddf['School'] = 'Stanford'

    
    


In [15]:
#fulldf = pd.concat([hbsdf, whartondf, stanforddf])

In [16]:
html = urlopen(str('https://aai.haas.berkeley.edu/scheduling/CourseSchedule.aspx?Semester=Spring&Year=2019'))
soup = BeautifulSoup(html, 'html.parser')


results = soup.find_all('span')

titlelist = []

for result in results:
    if len(result.text) > 4:
    
        titlelist.append(result.text)
            
linklist = []

results = soup.find_all('a', class_='blue')

for result in results:
    try:
        linklist.append(result['href'])
    
    except:
        continue        
        
        
bigdictlist = []
errorlist = []

for i in range(len(linklist)):
    
    try:
        html = urlopen(linklist[i])
        soup = BeautifulSoup(html, 'html.parser')
        interimlist = []
        
        for j in soup.find_all('p'):
            
            
            interimlist.append(j.text)
            
        bigdict = {linklist[i]: interimlist}
        
        
        bigdictlist.append(bigdict)
        
    except:
        errorlist.append(i)
        print(i)
        
        
testlist = []
desclist = []

for i in bigdictlist:
    for j in i[str(tuple(i)[0])]:
        if "COURSE TITLE" in j:
            testlist.append(j)
            desclist.append(i)

            
#not complete yet. However the above code does work now. The point of this is to cross reference back to
#bigdictlist from the actual course title. 

coursetitlelist = []

for j in range(len(bigdictlist)):
    for i in testlist:
        if i in bigdictlist[j][str(tuple(bigdictlist[j])[0])]:
            
            coursetitlelist.append(j)

            
#The numbers currently printing are the list indices from linklist which threw a 404 error when followed. Typing linklist[number] would give the link itself             

9
11
13
15
17
19
21
23
57
96
123
125
127
129
131
200
208
212


In [17]:
#desclist

In [18]:

haasdf = pd.DataFrame(testlist, columns= ['Course'])
haasdf['School'] = 'Haas'
haasdf['Source'] = ''
haasdf['Description'] = desclist
    

In [19]:
sourcelist = []

for i in range(len(desclist)):
    sourcelist.append(tuple(desclist[i])[0])
    
haasdf['Source']=sourcelist    

In [25]:
fulldf = pd.concat([hbsdf, whartondf, stanforddf, haasdf], sort=True)
partialdf = pd.concat([hbsdf, whartondf, stanforddf], sort=True)

partialdict = partialdf.to_dict(orient='list')
fulldict = fulldf.to_dict(orient='list')

In [27]:
db.curriculum.insert_one(partialdict)
#db.curriculum.insert_one(fulldict)

#Note: I inserted the partial dict without Haas. This is because of the Haas dictionary keys being URLs. I will fix this soon. Then fulldict can be inserted into mongoDB

#Note: full database should appear in mongoDB. Format for the 'Description' field is not consistent, but this is due to the underlying HTML of the pages and
#would be more trouble than it was worth to standardize. 

#Especially for Haas, this field will give a dictionary with the key being the URL and the values being the content. (This led to the error above. Will fix)
#This is because each description is a different URL, but some of these URLs led to 404 errors. 
#HBS has a similar structure, but it was possible to take only content in the "Description" field.


DuplicateKeyError: E11000 duplicate key error collection: curriculumDB.curriculum index: _id_ dup key: { : ObjectId('5c7586f7d4ee6419200c96e4') }

In [ ]:

#Note: Remaining code is unfinished and has been commented out. 

#source='http://www.tuck.dartmouth.edu/mba/academic-experience/elective-curriculum/elective-courses'

#html = urlopen(str(source))
#soup = BeautifulSoup(html, 'html.parser')

#divlist = []
#for i in soup.find_all("div", class_='row content' ):
 #   i.contents.p

In [ ]:
#divlist

In [ ]:
#soup.find_all('div',class_='row content')

In [ ]:
#This will have to wait until I have a more finished product to test on. 

#import pdfkit

#pdfkit.from_string(hbsdf, 'out.pdf')

In [ ]:
#In class, making notes for later. Can use .strip to remove whitespace. 

#look for class or id when you are doing .find_all as an argument within the function.